<a href="https://colab.research.google.com/github/Ismail-therap/NLP-Project-Predicting-review-in-1-to-5-star-/blob/main/CNN_5%2C_3_and_2_category_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Import libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

In [18]:
import pandas as pd

url='https://drive.google.com/file/d/1-vBpWV0H0cxX5jSeHlQIKXIiosMKwq2t/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
data = pd.read_csv(dwn_url)
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,0tGuFBrhM03I6TCLxDQhyg,Dancing Goats Coffee Bar,419 W Ponce De Leon Ave,Decatur,GA,30030,33.775805,-84.303269,4.0,358,1,"{'RestaurantsPriceRange2': '1', 'WiFi': ""'free...","Food, Coffee & Tea, Restaurants, Donuts, Cafes","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",IqUD0yoRq66PqnbSvRb5JA,1,OMG!! Hate! This place. Rude to me. On so many...,2018-10-06 14:34:51
1,498F3rnpng6qv53S9diTSw,Boston Market,"4607 Braker Ln, Ste 100",Austin,TX,78759,30.399832,-97.744997,3.0,102,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","American (Traditional), Caterers, Chicken Shop...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",n2eAlOnBo1St1p7MWqBFmg,1,Why bother being open on Thanksgiving if you c...,2019-11-28 20:02:11
2,7qBi3FMT2Oz4Jj1hCWwawA,Bea's Nail Salon,75 Blossom Ct,Boston,MA,02114,42.364891,-71.066818,3.0,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Nail Salons, Beauty & Spas","{'Monday': '9:30-19:0', 'Tuesday': '9:30-19:0'...",pHTrbVN4lzTlzqqnWEJYGw,1,"The fact that on a Saturday afternoon this ""sa...",2009-07-11 19:59:46
3,K4X4nJVp0BcSn9Cvoavk8A,T'Kilas Tacos & More,1248 Clairmont Rd,Decatur,GA,30030,33.789815,-84.307511,3.0,115,0,"{'GoodForKids': 'True', 'BikeParking': 'False'...","Mexican, Restaurants","{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",Jfc8qYR3uEHIpU0nKzpObg,1,"I came to Los Loros in search of the ""strong, ...",2008-09-08 21:34:50
4,D6U1_06gxciRs1NoYjwhKQ,ESPN Club - Temporarily Unavailable,2101 N Epcot Resorts Blvd,Orlando,FL,32836,28.368008,-81.555289,2.5,303,1,"{'GoodForKids': 'True', 'Caters': 'False', 'Re...","Hotels, American (New), Event Planning & Servi...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-23:0', ...",Dw_oka5MVWOsaJCncfIXZQ,1,My fries were stale and my cheesesteak was mos...,2018-01-14 01:41:50


# Five category

In [19]:
from sklearn.model_selection import train_test_split

data['five_star'] = data['review_stars'] - 1
review = data['text'].values
label = data['five_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[ 550  417  242    1 1543    1 1006   24   86 1265 1373    2 3166    1
 1006   24  528   86    1   68  212   13 2041    1 1006   24    1  143
   16 4350 1633 2921 2782  515 4428 4429 1606 3014    1  268   24   88
   21  474    4  256    2   15 1945    8   10    4 1059  905    9   21
 1846   31    7   47    7   12 1441    5   42 3811 1327  356 1595    7
  268  206  156    2 1658    6 1966  218    3  152   69 2254   15   74
 2198  715 1666  843 3445   49  534 3445   75 2921 2782  515 4428 4429
 1606 3014  178   44    3  289   17  102    9    4  230  550 2992   28
   26  550   10  741 2921 2782  515 4428 4429  550    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [20]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 450, 100)          5492700   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 450, 128)          51328     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 225, 128)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 225, 64)           32832     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 112, 64)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 112, 32)           8224      
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 56, 32)           

In [21]:
import time
start = time.time()


model.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)


print("Execution time: "+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Epoch 1/10
586/586 [==============================] - 122s 206ms/step - loss: 1.1395 - accuracy: 0.4810 - val_loss: 0.9260 - val_accuracy: 0.5931
Epoch 2/10
586/586 [==============================] - 106s 181ms/step - loss: 0.8677 - accuracy: 0.6191 - val_loss: 0.9292 - val_accuracy: 0.5902
Epoch 3/10
586/586 [==============================] - 119s 203ms/step - loss: 0.7700 - accuracy: 0.6625 - val_loss: 0.9397 - val_accuracy: 0.5894
Epoch 4/10
586/586 [==============================] - 119s 203ms/step - loss: 0.6752 - accuracy: 0.7096 - val_loss: 1.0055 - val_accuracy: 0.5846
Epoch 5/10
586/586 [==============================] - 119s 204ms/step - loss: 0.5638 - accuracy: 0.7659 - val_loss: 1.1547 - val_accuracy: 0.5782
Epoch 6/10
586/586 [==============================] - 119s 202ms/step - loss: 0.4411 - accuracy: 0.8199 - val_loss: 1.3904 - val_accuracy: 0.5634
Epoch 7/10
586/586 [==============================] - 118s 202ms/step - loss: 0.3245 - accuracy: 0.8727 - val_loss: 1.7676 -

# Three category

In [22]:
import numpy as np

# create a list of our conditions
conditions = [
    (data['review_stars'] <= 2),
    (data['review_stars'] == 3),
    (data['review_stars'] > 3)
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2]

# create a new column and use np.select to assign values to it using our lists as arguments
data['three_star'] = np.select(conditions, values)


# display updated DataFrame
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date,five_star,three_star
0,0tGuFBrhM03I6TCLxDQhyg,Dancing Goats Coffee Bar,419 W Ponce De Leon Ave,Decatur,GA,30030,33.775805,-84.303269,4.0,358,1,"{'RestaurantsPriceRange2': '1', 'WiFi': ""'free...","Food, Coffee & Tea, Restaurants, Donuts, Cafes","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",IqUD0yoRq66PqnbSvRb5JA,1,OMG!! Hate! This place. Rude to me. On so many...,2018-10-06 14:34:51,0,0
1,498F3rnpng6qv53S9diTSw,Boston Market,"4607 Braker Ln, Ste 100",Austin,TX,78759,30.399832,-97.744997,3.0,102,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","American (Traditional), Caterers, Chicken Shop...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",n2eAlOnBo1St1p7MWqBFmg,1,Why bother being open on Thanksgiving if you c...,2019-11-28 20:02:11,0,0
2,7qBi3FMT2Oz4Jj1hCWwawA,Bea's Nail Salon,75 Blossom Ct,Boston,MA,02114,42.364891,-71.066818,3.0,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Nail Salons, Beauty & Spas","{'Monday': '9:30-19:0', 'Tuesday': '9:30-19:0'...",pHTrbVN4lzTlzqqnWEJYGw,1,"The fact that on a Saturday afternoon this ""sa...",2009-07-11 19:59:46,0,0
3,K4X4nJVp0BcSn9Cvoavk8A,T'Kilas Tacos & More,1248 Clairmont Rd,Decatur,GA,30030,33.789815,-84.307511,3.0,115,0,"{'GoodForKids': 'True', 'BikeParking': 'False'...","Mexican, Restaurants","{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",Jfc8qYR3uEHIpU0nKzpObg,1,"I came to Los Loros in search of the ""strong, ...",2008-09-08 21:34:50,0,0
4,D6U1_06gxciRs1NoYjwhKQ,ESPN Club - Temporarily Unavailable,2101 N Epcot Resorts Blvd,Orlando,FL,32836,28.368008,-81.555289,2.5,303,1,"{'GoodForKids': 'True', 'Caters': 'False', 'Re...","Hotels, American (New), Event Planning & Servi...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-23:0', ...",Dw_oka5MVWOsaJCncfIXZQ,1,My fries were stale and my cheesesteak was mos...,2018-01-14 01:41:50,0,0


In [23]:
reviews_smallest = data[data["three_star"] == 1]

## sampling negative review:
reviews_neg = data[data["three_star"] == 0]
reviews_neg = reviews_neg.sample(n=reviews_smallest.shape[0])


## sampling positive review:
reviews_pos = data[data["three_star"] == 2]
reviews_pos = reviews_pos.sample(n=reviews_smallest.shape[0])

data = reviews_smallest.append([reviews_neg, reviews_pos])
data.shape

(30000, 20)

In [24]:
from sklearn.model_selection import train_test_split

#data['cnn_star'] = data['three_star'] - 1
review = data['text'].values
label = data['three_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[  19  595    1 1156   11   63   37 1093  967  190   11    4  161 2676
  662    2  232   22   19   64  595   16  229    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [25]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


#model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 450, 100)          4327300   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 450, 128)          51328     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 225, 128)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 225, 64)           32832     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 112, 64)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 112, 32)           8224      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 56, 32)           

In [26]:
import time
start = time.time()

model.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)


print("Execution time: "+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Epoch 1/10
352/352 [==============================] - 71s 200ms/step - loss: 0.8217 - accuracy: 0.5920 - val_loss: 0.6917 - val_accuracy: 0.6739
Epoch 2/10
352/352 [==============================] - 69s 197ms/step - loss: 0.6010 - accuracy: 0.7314 - val_loss: 0.6407 - val_accuracy: 0.7060
Epoch 3/10
352/352 [==============================] - 70s 199ms/step - loss: 0.4905 - accuracy: 0.7896 - val_loss: 0.6472 - val_accuracy: 0.7201
Epoch 4/10
352/352 [==============================] - 70s 200ms/step - loss: 0.3691 - accuracy: 0.8515 - val_loss: 0.7206 - val_accuracy: 0.7189
Epoch 5/10
352/352 [==============================] - 69s 197ms/step - loss: 0.2598 - accuracy: 0.8988 - val_loss: 0.9480 - val_accuracy: 0.6909
Epoch 6/10
352/352 [==============================] - 69s 197ms/step - loss: 0.1664 - accuracy: 0.9392 - val_loss: 1.1333 - val_accuracy: 0.6960
Epoch 7/10
352/352 [==============================] - 71s 201ms/step - loss: 0.1021 - accuracy: 0.9638 - val_loss: 1.4063 - val_ac

# Two category

In [27]:
import numpy as np

# create a list of our conditions
conditions = [
    (data['review_stars'] <= 2),
    (data['review_stars'] == 3),
    (data['review_stars'] > 3)
    ]

# create a list of the values we want to assign for each condition
values = [0,-1,1]

# create a new column and use np.select to assign values to it using our lists as arguments
data['two_star'] = np.select(conditions, values)


data = data[data.two_star != -1]

# display updated DataFrame
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes,categories,hours,user_id,review_stars,text,date,five_star,three_star,two_star
1528,naCBV24-FXHhscmohHwMDw,Extreme Pizza - Hillsboro,1888 NW 188th Ave,Hillsboro,OR,97006,45.534054,-122.870385,3.5,139,...,"{'RestaurantsDelivery': 'True', 'GoodForKids':...","Sandwiches, Gluten-Free, Pizza, Restaurants, S...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",4CK8q-p3FilvVLrkjhnczQ,1,Identity Theft issues? We have ordered from Ex...,2016-03-31 21:02:18,0,0,0
11175,ocHl-UFp5eu01v5H3L-Vbw,Northwest Oral And Facial Surgery,"6641 N High St, Ste 105",Columbus,OH,43085,40.099734,-83.018141,3.5,7,...,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Oral Surgeons, Dentists, Cosmetic Dentists, Ge...","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",lIyqkPMvyoQzvda7vs3SnA,2,I am not saying they didn't do the job. They d...,2019-08-15 03:31:36,1,0,0
13298,ifjN9JqXw6GNtcFJRLq8Kw,La Quinta Inn by Wyndham Austin Oltorf,1603 East Oltorf Blvd,Austin,TX,78741,30.232740,-97.740154,1.5,76,...,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Event Planning & Services, Hotels & Travel, Ho...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",nmYI1LmOEolCd1yidCYs1g,2,Stayed here on a Tuesday. The rooms we're dece...,2015-10-28 17:37:37,1,0,0
18873,gAze3qbX9r7Hjc1ciO44xw,Pappadeaux Seafood Kitchen,2830 Windy Hill Rd SE,Marietta,GA,30067,33.903148,-84.470340,4.0,1290,...,"{'HasTV': 'True', 'RestaurantsTakeOut': 'True'...","Southern, Seafood, Restaurants, Cajun/Creole","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",Qb6CdBfZlS9WOCJhxuvaVA,2,This location is subpar. Extremely colorful su...,2017-08-20 22:46:54,1,0,0
19943,3Pk2MFY2oeW12_mnI2x0Lg,H2O Hand Car Wash & Detail,505 W Slaughter Ln,Austin,TX,78748,30.170991,-97.798488,2.0,227,...,{'BusinessAcceptsCreditCards': 'True'},"Car Wash, Automotive, Auto Detailing","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",XyBFkTh2NB6IlINVpuHU4A,2,This place is awful. There is NO attention to ...,2014-09-27 17:52:07,1,0,0


In [28]:
from sklearn.model_selection import train_test_split

#data['cnn_star'] = data['three_star'] - 1
review = data['text'].values
label = data['two_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[   3   22   63    5 4008 2712    7  161   52 1535   52  227  133   11
    2    5   47 1058   36   21   22 4953 2221   14   60   35    7   52
  127    4 1762   70  558   52  522  571   10    1  636   11  155    1
   43    2   29   89 2132   93   63  403  944   29    9  232    2   11
   12   68  708   10  501    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [29]:
import tensorflow as tf

embedding_dim = 100
textcnnmodel = Sequential()
textcnnmodel.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(tf.keras.layers.Conv1D(200, 5, activation='relu',padding = 'same'))
#textcnnmodel.add(Dense(100, activation='relu'))
#textcnnmodel.add(tf.keras.layers.GlobalMaxPooling1D())
textcnnmodel.add(tf.keras.layers.Flatten())
textcnnmodel.add(tf.keras.layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 450, 100)          3556700   
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 450, 128)          51328     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 225, 128)          0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 225, 64)           32832     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 112, 64)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 112, 32)           8224      
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 56, 32)           

In [31]:
import time
start = time.time()

model.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 128)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)


print("Execution time: "+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Epoch 1/10
118/118 [==============================] - 46s 379ms/step - loss: 0.4410 - accuracy: 0.7694 - val_loss: 0.2073 - val_accuracy: 0.9184
Epoch 2/10
118/118 [==============================] - 44s 372ms/step - loss: 0.1569 - accuracy: 0.9426 - val_loss: 0.2446 - val_accuracy: 0.8984
Epoch 3/10
118/118 [==============================] - 44s 376ms/step - loss: 0.0958 - accuracy: 0.9667 - val_loss: 0.2622 - val_accuracy: 0.9104
Epoch 4/10
118/118 [==============================] - 44s 372ms/step - loss: 0.0581 - accuracy: 0.9800 - val_loss: 0.2560 - val_accuracy: 0.9180
Epoch 5/10
118/118 [==============================] - 44s 371ms/step - loss: 0.0270 - accuracy: 0.9918 - val_loss: 0.3311 - val_accuracy: 0.9184
Epoch 6/10
118/118 [==============================] - 44s 376ms/step - loss: 0.0167 - accuracy: 0.9944 - val_loss: 0.5087 - val_accuracy: 0.9008
Epoch 7/10
118/118 [==============================] - 45s 383ms/step - loss: 0.0220 - accuracy: 0.9920 - val_loss: 0.4254 - val_ac